# Extrator de informações genérico

In [1]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import re

## Criando lista de páginas

In [2]:
soups = []
files = []
for dirpath,_,filenames in os.walk(os.path.abspath('..\\Pages\\Positives\\')):
    for f in filenames:
        file = os.path.abspath(os.path.join(dirpath, f))
        files.append(file)
        soups.append(BeautifulSoup(
        open(file, encoding='utf-8'), 'html.parser'))
print(len(files))
len(soups)

91


91

## Extraindo nomes dos psicologos

Antes de começarmos vamos examinar rapidamente os campos h1's das páginas, uma vez que imaginasse que o nome do psicólogo deva ser encontrado neste campo

In [3]:
def clean_text(text):
    return text.replace('\n', '').replace('\t', '').strip();

for index, soup in enumerate(soups):
    h1s = soup.find_all('h1')
    if(len(h1s) == 1):
        print(str(index) + ' - ' + str(len(h1s)) + ' -- ' + clean_text(h1s[0].text))
    elif(len(h1s) == 2):
        print(str(index) + ' - ' + str(len(h1s)) + ' - ' + str(h1s[0].parent.name) +
              ' - ' + clean_text(h1s[0].text) + ' -- ' + str(h1s[1].parent.name) + ' - '+ clean_text(h1s[1].text))
    else:
        print( str(index) + ' - Empty result')



0 - Empty result
1 - Empty result
2 - Empty result
3 - Empty result
4 - Empty result
5 - Empty result
6 - Empty result
7 - Empty result
8 - Empty result
9 - Empty result
10 - 2 - div - Sérgio Alexandre Alves Fernandes -- div - Sérgio Alexandre Alves Fernandes
11 - 2 - div - Luiz Gustavo Canuto -- div - Luiz Gustavo Canuto
12 - 2 - div - Igor Costa -- div - Igor Costa
13 - 2 - div - Bruno Schievenin Madi -- div - Bruno Schievenin Madi
14 - 2 - div - Sandro Azevêdo -- div - Sandro Azevêdo
15 - 2 - div - Wilza M Brosig -- div - Wilza M Brosig
16 - 2 - div - Dra. Luciana Dias Nascimento -- div - Dra. Luciana Dias Nascimento
17 - 2 - div - Fernanda Simionato Costa -- div - Fernanda Simionato Costa
18 - 2 - div - Rodrigo Noia Mattos Montan -- div - Rodrigo Noia Mattos Montan
19 - 2 - div - Kassia Arouca cardoso -- div - Kassia Arouca cardoso
20 - 2 - div - Dr. Caio Fasoli Rebouças -- div - Dr. Caio Fasoli Rebouças
21 - 1 -- ADRIANA PANZAN
22 - 1 -- CONSULTORIO DE PSICOLOGIA SORAIA S GRANADO 

Analisando a coleta acima é possível perceber que existem poucos padrões diferentes.

Primeiro podemos identificar que as páginas possuem de 0 a 2 campos com a tag H1.

Onde o classificados não possui uma tag h1 em seu corpo, enquanto o doctoralia e o telelistas possuem essa tag 2x em seu corpo, além disto o dominio <b>psicolink</b> possui 1 página outlier que possui 2 tags H1 na mesma página, enquanto todas as demais possuem apenas 1.

A maioria dos dominios pode se utilizar apenas pegar o primeiro campo H1 para se extrair o campo nome, porém os dominios Psicólgoos do Brasil, telelistas e classificados precisam de um tratamento extra para serem cobertos.

Segue abaixo suas particularidades.
O dominio <b>classificados</b> não possui a tag h1, onde é possível observar que seus campos estão sempre dentro de uma tabela, talvez esse domínio seja o mais diferente dentre os utilizados.

O <b>telelistas</b> é o único cuja tag <b>H1</b> esta dentro de uma tag <b>A</b> e esta deve ser ignorada, também sendo o único em que o campo nome não esta na primeira tag <b>H1</b> encontrada e ainda apesar do campo nome se encontrar na segunda tag <b>H1</b> o mesmo vem com a cidade pós-fixada dentro do próprio campo.

O domínio <b>psicólogos do brasil</b> possui o campo nome na tag <b>H1</b>, porém o mesmo trás como prefixo o nome do próprio domínio, enquanto o nome correto sem o domínio, se encontra em uma tag h2.

Assim com o algoritmo de extrair o campo diretamente da primeira tag <b>H1</b> encontrada, possuimos:

<b>Cobertura: 89%</b>
<b>Precisão: 75.3%</b>

### Melhorando a extração dos nomes
Desta forma a primeira estratégia que se vem a mente para aumentar a precisão do extrator genérico é adicionar condicionais a cerca da seleção da tag quando encontrarmos mais que uma tag <b>H1</b> na página, ao simplesmente alterar a estratégia para pegar a última tag <b>H1</b>, obtemos uma <b>Cobertura de 89% </b> e <b>Precisão de 84.5%</b> pois coletaremos corretamente todas as páginas do domínio <b>Telelistas</b>, porém o outlier encontrado no domínio <b>psicolink</b> não será extraido corretamente.

In [4]:
for index, soup in enumerate(soups):
    h1s = soup.find_all('h1')
    if(len(h1s) == 1):
        print(str(index) + '-' + clean_text(h1s[0].text))
    elif(len(h1s) == 2):
        print(str(index) + '-' + clean_text(h1s[1].text))
    else:
        print( str(index) + '-Empty result')

0-Empty result
1-Empty result
2-Empty result
3-Empty result
4-Empty result
5-Empty result
6-Empty result
7-Empty result
8-Empty result
9-Empty result
10-Sérgio Alexandre Alves Fernandes
11-Luiz Gustavo Canuto
12-Igor Costa
13-Bruno Schievenin Madi
14-Sandro Azevêdo
15-Wilza M Brosig
16-Dra. Luciana Dias Nascimento
17-Fernanda Simionato Costa
18-Rodrigo Noia Mattos Montan
19-Kassia Arouca cardoso
20-Dr. Caio Fasoli Rebouças
21-ADRIANA PANZAN
22-CONSULTORIO DE PSICOLOGIA SORAIA S GRANADO RODRIGUES
23-PSICOVIDA - PSICOLOGIA & NEUROPSICOLOGIA
24-SIMONE GUIMARAES PSICOLOGIA
25-ANA MEDIOLARO PSICOLOGIA CLINICA
26-DANIELLE CRISTINA GAZZONI
27-DR. ADRIANO MACHADO OLIVEIRA - SALUTE 21
28-LEILA  DOURADO
29-ESPAÇO COMPORTAMENTAL
30-ANDREA ZANETTI
31-Mônica Komora
32-Abdon Sardinha
33-Espaço Consciência Psicologia e outras Terapias
34-Sandra Pereira
35-Vera Pelizzari Psicóloga Clínica
36-Novos Horizontes
37-Ussénade Maria de Oliveira
38-Sumara Gonçalves
39-Cristyani Paiva
40-Lia Soares Dantas
41-H

Com um pequeno processamento extra para remover o que se encontra antes de um <b> - </b> podemos melhorar ainda mais a precisão.
Porém existe uma chance de isto ser considerado um overfitting e poder errar quando novos dominios forem adicionados. Com esta modificação o a precisão aumenta para <b> 97.5%</b>, manténdo os <b>84.5%</b> de cobertura

In [5]:
def clean_text(text):
    start = max(0,text.find('- '))
    if(start >0):
        start = start + 2
    text = text[start:]
    return text.replace('\n', '').replace('\t', '').strip();

for index, soup in enumerate(soups):
    h1s = soup.find_all('h1')
    if(len(h1s) == 1):
        print(str(index) + '-' + clean_text(h1s[0].text))
    elif(len(h1s) == 2):
        print(str(index) + '-' + clean_text(h1s[1].text))
    else:
        print( str(index) + '-Empty result')


0-Empty result
1-Empty result
2-Empty result
3-Empty result
4-Empty result
5-Empty result
6-Empty result
7-Empty result
8-Empty result
9-Empty result
10-Sérgio Alexandre Alves Fernandes
11-Luiz Gustavo Canuto
12-Igor Costa
13-Bruno Schievenin Madi
14-Sandro Azevêdo
15-Wilza M Brosig
16-Dra. Luciana Dias Nascimento
17-Fernanda Simionato Costa
18-Rodrigo Noia Mattos Montan
19-Kassia Arouca cardoso
20-Dr. Caio Fasoli Rebouças
21-ADRIANA PANZAN
22-CONSULTORIO DE PSICOLOGIA SORAIA S GRANADO RODRIGUES
23-PSICOLOGIA & NEUROPSICOLOGIA
24-SIMONE GUIMARAES PSICOLOGIA
25-ANA MEDIOLARO PSICOLOGIA CLINICA
26-DANIELLE CRISTINA GAZZONI
27-SALUTE 21
28-LEILA  DOURADO
29-ESPAÇO COMPORTAMENTAL
30-ANDREA ZANETTI
31-Mônica Komora
32-Abdon Sardinha
33-Espaço Consciência Psicologia e outras Terapias
34-Sandra Pereira
35-Vera Pelizzari Psicóloga Clínica
36-Novos Horizontes
37-Ussénade Maria de Oliveira
38-Sumara Gonçalves
39-Cristyani Paiva
40-Lia Soares Dantas
41-Hildes do Amparo Delduque Farina
42-Silvia R

In [6]:
names = []

for index, soup in enumerate(soups):
    h1s = soup.find_all('h1')
    if(len(h1s) == 1):
        names.append(clean_text(h1s[0].text))
    elif(len(h1s) == 2):
        names.append(clean_text(h1s[1].text))
    else:
        print( str(index) + '-Empty result')
        names.append('')

d = {'names': names}
df = pd.DataFrame(data=d)
df

0-Empty result
1-Empty result
2-Empty result
3-Empty result
4-Empty result
5-Empty result
6-Empty result
7-Empty result
8-Empty result
9-Empty result


,names
0,
1,
2,
3,
4,
5,
6,
7,
8,
9,


## Coletando telefones

In [142]:
phones = []
primary_phone = []

def clean_text(text):
    return text.replace('\n', '').replace('\t', '').strip().encode('ascii',errors='ignore').decode()

def remove_prefix(text):
    position = text.find(':')+1
    return text[position:].strip()

def is_phone(text):
    separator_position = text.find('-')
    result = text[0]=='(' or ( separator_position > 0 and len(text[separator_position:])>4 )
#    result = result or text.find(' ') > 0
    print(text)
    return result

def choice_result(texts):
    result = []
    for text in texts:
        cleaned_text = clean_text(text[0])
        cleaned_text = remove_prefix(cleaned_text)
        if(is_phone(cleaned_text)):
            result.append(cleaned_text)
    print(result)
    return str(result)

#re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}",s)
regex = r"(\w[\w\d_]*:\s*(\(\d*\))?(\d*)?\s*\d{3,}-?\d+)"
rg = re.compile(regex)

for index, soup in enumerate(soups):
    test_str = soup.text

    result = rg.findall(test_str)
    if result:
        print(str(index) + ' ' + str(len(result)) + str(result))
        result = choice_result(result)
        phones.append(result)
    else:
        phones.append('')


0 2[('Contato: \xa0\n11 996970779', '', '11'), ('Telefone:\n1128632313', '', '112863')]
11 996970779
1128632313
[]
1 2[('Contato: \xa0\n(34)99880-6090', '(34)', '99'), ('Telefone:\n(34)99880-6090', '(34)', '99')]
(34)99880-6090
(34)99880-6090
['(34)99880-6090', '(34)99880-6090']
2 1[('Telefone:\n999386868', '', '99938')]
999386868
[]
3 2[('Contato: \xa0\n981343194', '', '98134'), ('Telefone:\n981453194', '', '98145')]
981343194
981453194
[]
4 2[('Fones: 2545-1075', '', '2'), ('Telefone:\n2545-1075', '', '2')]
2545-1075
2545-1075
['2545-1075', '2545-1075']
5 1[('Telefone:\n21979402189', '', '2197940')]
21979402189
[]
6 1[('Telefone:\n(51)98004640', '(51)', '9800')]
(51)98004640
['(51)98004640']
7 2[('Contato: \xa0\n11 99567-5258', '', '11'), ('Telefone:\n11 99567-5258', '', '11')]
11 99567-5258
11 99567-5258
['11 99567-5258', '11 99567-5258']
8 2[('Contato: \xa0\n11 99567-5258', '', '11'), ('Telefone:\n11 99567-5258', '', '11')]
11 99567-5258
11 99567-5258
['11 99567-5258', '11 99567-52

73 55[('index:1070', '', ''), ('index:99999', '', '9'), ('index:99999', '', '9'), ('index:99999', '', '9'), ('index:9999999', '', '999'), ('index:1250', '', ''), ('width:1150', '', ''), ('index:99999', '', '9'), ('index:1250', '', ''), ('width:1024', '', ''), ('index:1030', '', ''), ('width:1440', '', ''), ('width:1441', '', ''), ('width:1024', '', ''), ('width:1024', '', ''), ('width:1024', '', ''), ('index:9999', '', ''), ('index:999999', '', '99'), ('width:1200', '', ''), ('width:1441', '', ''), ('Date: 2018-01', '', '2'), ('width:1200', '', ''), ('width:1441', '', ''), ('width:1199', '', ''), ('width:1199', '', ''), ('width:1150', '', ''), ('index:9999999', '', '999'), ('index:9999', '', ''), ('width:1024', '', ''), ('width:1200', '', ''), ('width:1025', '', ''), ('width:1025', '', ''), ('index:2147483648', '', '214748'), ('xl:1200', '', ''), ('width:1200', '', ''), ('width:1140', '', ''), ('width:1200', '', ''), ('width:1199', '', ''), ('index:1000', '', ''), ('width:1199', '', ''

79 55[('index:1070', '', ''), ('index:99999', '', '9'), ('index:99999', '', '9'), ('index:99999', '', '9'), ('index:9999999', '', '999'), ('index:1250', '', ''), ('width:1150', '', ''), ('index:99999', '', '9'), ('index:1250', '', ''), ('width:1024', '', ''), ('index:1030', '', ''), ('width:1440', '', ''), ('width:1441', '', ''), ('width:1024', '', ''), ('width:1024', '', ''), ('width:1024', '', ''), ('index:9999', '', ''), ('index:999999', '', '99'), ('width:1200', '', ''), ('width:1441', '', ''), ('Date: 2018-01', '', '2'), ('width:1200', '', ''), ('width:1441', '', ''), ('width:1199', '', ''), ('width:1199', '', ''), ('width:1150', '', ''), ('index:9999999', '', '999'), ('index:9999', '', ''), ('width:1024', '', ''), ('width:1200', '', ''), ('width:1025', '', ''), ('width:1025', '', ''), ('index:2147483648', '', '214748'), ('xl:1200', '', ''), ('width:1200', '', ''), ('width:1140', '', ''), ('width:1200', '', ''), ('width:1199', '', ''), ('index:1000', '', ''), ('width:1199', '', ''

In [143]:
d = {'names':names, 'phones': phones}
df = pd.DataFrame(data=d)
df

,names,phones
0,,[]
1,,"['(34)99880-6090', '(34)99880-6090']"
2,,[]
3,,[]
4,,"['2545-1075', '2545-1075']"
5,,[]
6,,['(51)98004640']
7,,"['11 99567-5258', '11 99567-5258']"
8,,"['11 99567-5258', '11 99567-5258']"
9,,['(48)34654498']
